In [102]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
import spacy

In [103]:
nlp = spacy.load("en_core_web_sm")

# Define URLs for articles
urls = [
    "https://bleacherreport.com/articles/10077241-every-teams-biggest-need-in-the-2023-nba-draft",
    "https://www.skysports.com/nba/news/36226/12903559/2023-nba-draft-team-needs-for-every-eastern-conference-franchise",
    "https://en.as.com/nba/2023-nba-draft-team-by-team-analysis-projected-picks-and-needs-for-rebuilding-n",
    "https://www.noceilingsnba.com/p/2023-nba-draft-gradeswell-sort-of",
    "https://www.bulletsforever.com/2023/6/16/23762624/the-yoda-guide-to-the-2023-nba-draft",
    "https://www.reddit.com/r/NBA_Draft/comments/13sl7ex/2023_nba_draft_prospects_comprehensive_spreadsheet",
    "https://www.nytimes.com/athletic/4138828/2023/01/31/nba-trade-deadline-biggest-needs/",
    "https://www.espn.com/nba/insider/story/_/id/37919086/nba-free-agency-2023-priorities-needs-all-30-teams",
    "https://www.fearthesword.com/2023/6/22/23767139/2023-nba-draft-preview-cleveland-cavaliers",
    "https://www.blazersedge.com/2024/6/26/24186994/trail-blazers-2024-nba-draft-grade-reaction-avdija-clingan-trades"
]

# Fetch pages and parse with BeautifulSoup
articles = []
titles = []

for url in urls:
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Extract title
        title = soup.find("h1")
        titles.append(title.get_text(strip=True) if title else "No Title Found")

        # Extract content
        paragraphs = soup.find_all("p")
        text = ' '.join([p.get_text() for p in paragraphs])
        articles.append(text)
    else:
        titles.append("Failed to Fetch Article")
        articles.append("")

# Print titles to confirm
for i, title in enumerate(titles, start=1):
    print(f"Article {i} Title: {title}")

Article 1 Title: Every Team's Biggest Need In the 2023 NBA Draft
Article 2 Title: 2023 NBA Draft: Team needs for every Eastern Conference franchise
Article 3 Title: 2023 NBA draft: Team-by-team analysis, projected picks, and needs for rebuilding
Article 4 Title: 
Article 5 Title: The YODA Guide to the 2023 NBA Draft
Article 6 Title: 2023 NBA Draft Prospects Comprehensive Spreadsheet
Article 7 Title: NBA trade deadline 2023: Identifying biggest need for all 30 teams
Article 8 Title: Failed to Fetch Article
Article 9 Title: 2023 NBA Draft Preview: Can the Cleveland Cavaliers find creative solutions?
Article 10 Title: Blazers Address Needs, Add Flexibility on Draft Night


In [104]:
import spacy
import re
from collections import defaultdict

In [105]:
# Define NBA team names
teams = [
    "Hawks", "Celtics", "Nets", "Hornets", "Bulls", "Cavaliers", "Mavericks", "Nuggets",
    "Pistons", "Warriors", "Rockets", "Pacers", "Clippers", "Lakers", "Grizzlies", "Heat",
    "Bucks", "Timberwolves", "Pelicans", "Knicks", "Thunder", "Magic", "76ers", "Sixers", "Suns",
    "Trail Blazers", "Kings", "Spurs", "Raptors", "Jazz", "Wizards"
]

# Define keywords indicating team needs
team_needs_keywords = {
    "looking for", "seeking", "need", "targeting", "hoping to add", "want", "desperate for", 
    "interested in", "pursuing", "eyeing", "trying to acquire", "desiring", "focused on", 
    "priority for", "in need of", "attempting to strengthen", "looking to improve", 
    "looking to upgrade", "trying to bolster", "dealing for", "acquiring", "hoping to land", 
    "focused on improving", "in pursuit of", "wants to trade for", "priority is", "desperately seeking",
    
    # Specific position/role keywords
    "shooter", "point guard", "big man", "center", "forward", "shooting guard", "wing", 
    "playmaker", "scorer", "defender", "rim protector", "rebounder", "backup", "bench scorer", 
    "ball handler", "stretch four", "floor spacer", "three-point shooter", "rim protector", 
    "athletic guard", "versatile forward", "defensive stopper"
}

# Dictionary to store extracted mentions per team
team_mentions = defaultdict(list)

# Process each article
for i, article in enumerate(articles, start=1):
    if not article.strip():  # Skip empty articles
        print(f"Skipping Article {i}, as it's empty.")
        continue

    doc = nlp(article)  # Process with spaCy
    
    for sent in doc.sents:  # Iterate through sentences
        sentence_text = sent.text
        for team in teams:
            if team in sentence_text and any(keyword in sentence_text for keyword in team_needs_keywords):
                team_mentions[team].append(f"Article {i}: {sentence_text}")

# Display extracted mentions
for team, mentions in team_mentions.items():
    print(f"\n**{team} Mentions:**")
    for mention in set(mentions):  # Use set to remove duplicates
        print(f" - {mention}")

Skipping Article 8, as it's empty.

**Hawks Mentions:**
 - Article 2: The Hawks are all-in on Trae Young so they need to find some defence around him and some more shooters that he can feed with his crafty playmaking skills.
 - Article 1: As long as the Hawks remain committed to Trae Young, they need to tailor their roster in a way that emphasizes his strengths and masks his weaknesses.
 - Article 1: Adding AJ Griffin at last year's draft was a start, but the Hawks need to keep pushing.
 - Article 1: Those aren't anywhere near where they'd need to be for the Hawks to contend.

**Celtics Mentions:**
 - Article 7: Luke Kornet has been solid, but the Celtics could use another center who can play a different style than he does.
 - Article 7: The Celtics need more bench scoring when Brogdon isn’t catching fire, and another forward can help.
 - Article 1: In our reality, where the Celtics hold only the No. 35 pick, they might be thrilled to snag a center who checks two of those boxes.
 - Art

In [182]:
import pandas as pd
import re
from collections import defaultdict, Counter

# Define mapping of keywords to position categories
position_mapping = {
    "point guard": "PG", "shooting guard": "SG", "big man": "PF/C", "center": "C", "forward": "SF/PF",
    "wing": "SG/SF", "playmaker": "PG/SG", "scorer": "SG/SF", "defender": "SF/PF", "rim protector": "C",
    "rebounder": "PF/C", "backup": "PG", "bench scorer": "SG", "ball handler": "PG",
    "stretch four": "PF", "floor spacer": "SF", "three-point shooter": "SG", "athletic guard": "SG",
    "versatile forward": "SF/PF", "defensive stopper": "SF/PF"
}

# Define "Needs" categories based on team strategies
needs_mapping = {
    "shooter": "Shooter/Upside", "point guard": "Playmaker", "big man": "NBA-ready", "center": "Defensive Center",
    "forward": "Defensive Forward", "shooting guard": "Scoring Guard", "wing": "3 and D",
    "playmaker": "Playmaker", "scorer": "Scoring Guard", "defender": "Defensive Stopper",
    "rim protector": "Defensive Center", "rebounder": "Defensive Versatile Forward",
    "backup": "Backup Guard", "bench scorer": "Bench Scorer", "ball handler": "Playmaker",
    "stretch four": "3 and D Forward", "floor spacer": "3 and D Forward", "three-point shooter": "Shooter",
    "athletic guard": "Upside Guard", "versatile forward": "Upside Forward",
    "defensive stopper": "Defensive Stopper"
}

# Function to convert summary into structured data
def refine_summary(summary):
    pos_needed = position_mapping.get(summary, "NaN")  # Map to a structured position
    needs = needs_mapping.get(summary, "Young Talent/Upside")  # Assign needs category
    return pos_needed, needs

# Create structured DataFrame
data = []
for team, mentions in team_mentions.items():
    summary = summarize_mentions(mentions)
    pos_needed, needs = refine_summary(summary)
    data.append([team, pos_needed, needs])

df = pd.DataFrame(data, columns=["Team", "Pos_needed", "Needs"])

# Display final structured DataFrame
print(df)

            Team Pos_needed                        Needs
0          Hawks        NaN          Young Talent/Upside
1        Celtics          C             Defensive Center
2        Rockets      SF/PF            Defensive Forward
3          Bulls        NaN          Young Talent/Upside
4        Nuggets      SG/SF                      3 and D
5         Pacers      SG/SF                      3 and D
6         Lakers        NaN          Young Talent/Upside
7      Grizzlies      SG/SF                      3 and D
8           Heat        NaN          Young Talent/Upside
9         Sixers      SF/PF            Defensive Stopper
10         Kings        NaN          Young Talent/Upside
11         Spurs      SG/SF                      3 and D
12       Raptors      SG/SF                      3 and D
13          Jazz        NaN          Young Talent/Upside
14       Wizards        NaN          Young Talent/Upside
15          Nets       PF/C  Defensive Versatile Forward
16       Hornets      SF/PF    

In [150]:
df.to_csv('2025teamneeds', index = False)

In [218]:
import pandas as pd
import numpy as np

# Current dataset with missing values
current_data = pd.read_csv('/Users/ethantsao/NBADraft1/2025teamneeds')

# Data from 2023
data_2023 = pd.read_csv('/Users/ethantsao/NBADraft1/2023teamneeds')

# Data from 2024
data_2024 = pd.read_csv('/Users/ethantsao/NBADraft1/2024teamneeds')

# Combine both historical datasets for better pattern recognition
historical_data = pd.concat([data_2023, data_2024])

# Function to analyze patterns and predict position needs
def predict_positions(current_df, historical_df):
    # Create a copy of the current dataframe
    result_df = current_df.copy()
    
    # Dictionary to store patterns from historical data
    need_to_pos_mapping = {}
    
    # Build the mapping from historical data
    for need in historical_df['Needs'].unique():
        positions = historical_df[historical_df['Needs'] == need]['Pos_needed'].dropna().tolist()
        if positions:
            # Get the most common position for this need
            pos_counts = {}
            for pos in positions:
                pos_counts[pos] = pos_counts.get(pos, 0) + 1
            most_common_pos = max(pos_counts, key=pos_counts.get)
            need_to_pos_mapping[need] = most_common_pos
    
    # Fill missing positions based on the need-to-position mapping
    for idx, row in result_df.iterrows():
        if pd.isna(row['Pos_needed']) and row['Needs'] in need_to_pos_mapping:
            result_df.at[idx, 'Pos_needed'] = need_to_pos_mapping[row['Needs']]
    
    return result_df

# Predict positions based on needs
predicted_data = predict_positions(current_data, historical_data)

# Display the result with filled positions
print("Predicted NBA Team Position Needs:")
for idx, row in predicted_data.iterrows():
    print(f"{row['Team']}: {row['Pos_needed']} - {row['Needs']}")

# Order teams according to the draft order provided
draft_order = [
    'Wizards', 'Hornets', 'Jazz', 'Pelicans', 'Raptors', 'Sixers', 'Nets', 'Bulls',
    'Spurs', 'Trail Blazers', 'Rockets', 'Spurs', 'Hawks', 'Mavericks', 'Thunder', 'Magic',
    'Jazz', 'Heat', 'Thunder', 'Timberwolves', 'Nets', 'Pacers', 'Nets', 'Hawks',
    'Magic', 'Wizards', 'Nets', 'Celtics', 'Clippers', 'Suns'
]

# Create a mapping of teams to their predicted positions and needs
team_to_info = {row['Team']: (row['Pos_needed'], row['Needs']) for _, row in predicted_data.iterrows()}

# Create the final dataframe in draft order
ordered_data = []
for team in draft_order:
    if team in team_to_info:
        pos_needed, needs = team_to_info[team]
        ordered_data.append({'Team': team, 'Pos_needed': pos_needed, 'Needs': needs})
    else:
        # Handle teams not in the original dataframe (like Trail Blazers)
        ordered_data.append({'Team': team, 'Pos_needed': None, 'Needs': None})

# Create the final ordered dataframe
final_ordered_df = pd.DataFrame(ordered_data)

# Print the final draft order with positions and needs
print("\nFinal NBA Draft Order with Position Needs:")

Predicted NBA Team Position Needs:
Hawks: SF/PF - Young Talent/Upside
Celtics: C - Defensive Center
Rockets: SF/PF - Defensive Forward
Bulls: SF/PF - Young Talent/Upside
Nuggets: SG/SF - 3 and D
Pacers: SG/SF - 3 and D
Lakers: SF/PF - Young Talent/Upside
Grizzlies: SG/SF - 3 and D
Heat: SF/PF - Young Talent/Upside
Sixers: SF/PF - Defensive Stopper
Kings: SF/PF - Young Talent/Upside
Spurs: SG/SF - 3 and D
Raptors: SG/SF - 3 and D
Jazz: SF/PF - Young Talent/Upside
Wizards: SF/PF - Young Talent/Upside
Nets: PF/C - Defensive Versatile Forward
Hornets: SF/PF - Defensive Forward
Magic: SG/SF - 3 and D
Clippers: C - Defensive Center
Pistons: SF/PF - Defensive Forward
Mavericks: SF/PF - Young Talent/Upside
Thunder: SG/SF - 3 and D
Warriors: SF/PF - Young Talent/Upside
Suns: SF/PF - Young Talent/Upside
Bucks: SG/SF - 3 and D
Pelicans: SF - Shooter/Upside
Knicks: SF/PF - Young Talent/Upside
Timberwolves: PG/SG - Playmaker
Cavaliers: SF/PF - Young Talent/Upside

Final NBA Draft Order with Positio

In [226]:
import pandas as pd
import numpy as np

# Create the full draft order with 59 picks
draft_picks = [
    # 1st Round
    {'Pick': 1, 'Team': 'Washington', 'Original_Team': 'Washington'},
    {'Pick': 2, 'Team': 'Charlotte', 'Original_Team': 'Charlotte'},
    {'Pick': 3, 'Team': 'Utah', 'Original_Team': 'Utah'},
    {'Pick': 4, 'Team': 'New Orleans', 'Original_Team': 'New Orleans'},
    {'Pick': 5, 'Team': 'Toronto', 'Original_Team': 'Toronto'},
    {'Pick': 6, 'Team': 'Philadelphia', 'Original_Team': 'Philadelphia'},
    {'Pick': 7, 'Team': 'Brooklyn', 'Original_Team': 'Brooklyn'},
    {'Pick': 8, 'Team': 'Chicago', 'Original_Team': 'Chicago'},
    {'Pick': 9, 'Team': 'San Antonio', 'Original_Team': 'San Antonio'},
    {'Pick': 10, 'Team': 'Portland', 'Original_Team': 'Portland'},
    {'Pick': 11, 'Team': 'Houston', 'Original_Team': 'PHX'},
    {'Pick': 12, 'Team': 'San Antonio', 'Original_Team': 'ATL'},
    {'Pick': 13, 'Team': 'Atlanta', 'Original_Team': 'SAC'},
    {'Pick': 14, 'Team': 'Dallas', 'Original_Team': 'Dallas'},
    {'Pick': 15, 'Team': 'Oklahoma City', 'Original_Team': 'MIA'},
    {'Pick': 16, 'Team': 'Orlando', 'Original_Team': 'Orlando'},
    {'Pick': 17, 'Team': 'Utah', 'Original_Team': 'MIN'},
    {'Pick': 18, 'Team': 'Miami', 'Original_Team': 'GS'},
    {'Pick': 19, 'Team': 'Oklahoma City', 'Original_Team': 'LAC'},
    {'Pick': 20, 'Team': 'Minnesota', 'Original_Team': 'DET'},
    {'Pick': 21, 'Team': 'Brooklyn', 'Original_Team': 'MIL'},
    {'Pick': 22, 'Team': 'Indiana', 'Original_Team': 'Indiana'},
    {'Pick': 23, 'Team': 'Brooklyn', 'Original_Team': 'HOU'},
    {'Pick': 24, 'Team': 'Atlanta', 'Original_Team': 'LAL'},
    {'Pick': 25, 'Team': 'Orlando', 'Original_Team': 'DEN'},
    {'Pick': 26, 'Team': 'Washington', 'Original_Team': 'MEM'},
    {'Pick': 27, 'Team': 'Brooklyn', 'Original_Team': 'NY'},
    {'Pick': 28, 'Team': 'Boston', 'Original_Team': 'Boston'},
    {'Pick': 29, 'Team': 'LA Clippers', 'Original_Team': 'OKC'},
    {'Pick': 30, 'Team': 'Phoenix', 'Original_Team': 'CLE'},
    
    # 2nd Round
    {'Pick': 31, 'Team': 'Boston', 'Original_Team': 'WAS'},
    {'Pick': 32, 'Team': 'Minnesota', 'Original_Team': 'UTA'},
    {'Pick': 33, 'Team': 'Charlotte', 'Original_Team': 'Charlotte'},
    {'Pick': 34, 'Team': 'Charlotte', 'Original_Team': 'NO'},
    {'Pick': 35, 'Team': 'Detroit', 'Original_Team': 'TOR'},
    {'Pick': 36, 'Team': 'Philadelphia', 'Original_Team': 'Philadelphia'},
    {'Pick': 37, 'Team': 'Brooklyn', 'Original_Team': 'Brooklyn'},
    {'Pick': 38, 'Team': 'Sacramento', 'Original_Team': 'CHI'},
    {'Pick': 39, 'Team': 'San Antonio', 'Original_Team': 'San Antonio'},
    {'Pick': 40, 'Team': 'Toronto', 'Original_Team': 'POR'},
    {'Pick': 41, 'Team': 'Oklahoma City', 'Original_Team': 'ATL'},
    {'Pick': 42, 'Team': 'Washington', 'Original_Team': 'PHX'},
    {'Pick': 43, 'Team': 'Golden State', 'Original_Team': 'MIA'},
    {'Pick': 44, 'Team': 'Orlando', 'Original_Team': 'Orlando'},
    {'Pick': 45, 'Team': 'Chicago', 'Original_Team': 'SAC'},
    {'Pick': 46, 'Team': 'LA Clippers', 'Original_Team': 'MIN'},
    {'Pick': 47, 'Team': 'Utah', 'Original_Team': 'DAL'},
    {'Pick': 48, 'Team': 'Washington', 'Original_Team': 'GS'},
    {'Pick': 49, 'Team': 'Utah', 'Original_Team': 'LAC'},
    {'Pick': 50, 'Team': 'Washington', 'Original_Team': 'DET'},
    {'Pick': 51, 'Team': 'Cleveland', 'Original_Team': 'MIL'},
    {'Pick': 52, 'Team': 'Indiana', 'Original_Team': 'Indiana'},
    {'Pick': 53, 'Team': 'Memphis', 'Original_Team': 'HOU'},
    {'Pick': 54, 'Team': 'LA Lakers', 'Original_Team': 'LA Lakers'},
    {'Pick': 55, 'Team': 'Phoenix', 'Original_Team': 'DEN'},
    {'Pick': 56, 'Team': 'New York', 'Original_Team': 'MEM'},
    {'Pick': 57, 'Team': 'Orlando', 'Original_Team': 'BOS'},
    {'Pick': 58, 'Team': 'Houston', 'Original_Team': 'OKC'},
    {'Pick': 59, 'Team': 'Cleveland', 'Original_Team': 'Cleveland'}
]

# Create DataFrame from the draft picks
draft_df = pd.DataFrame(draft_picks)

# Current dataset with known position needs
current_data = {
    'Hawks': {'Pos_needed': np.nan, 'Needs': 'Young Talent/Upside'},
    'Celtics': {'Pos_needed': 'C', 'Needs': 'Defensive Center'},
    'Rockets': {'Pos_needed': 'SF/PF', 'Needs': 'Defensive Forward'},
    'Bulls': {'Pos_needed': np.nan, 'Needs': 'Young Talent/Upside'},
    'Nuggets': {'Pos_needed': 'SG/SF', 'Needs': '3 and D'},
    'Pacers': {'Pos_needed': 'SG/SF', 'Needs': '3 and D'},
    'Lakers': {'Pos_needed': np.nan, 'Needs': 'Young Talent/Upside'},
    'Grizzlies': {'Pos_needed': 'SG/SF', 'Needs': '3 and D'},
    'Heat': {'Pos_needed': np.nan, 'Needs': 'Young Talent/Upside'},
    'Sixers': {'Pos_needed': 'SF/PF', 'Needs': 'Defensive Stopper'},
    'Kings': {'Pos_needed': np.nan, 'Needs': 'Young Talent/Upside'},
    'Spurs': {'Pos_needed': 'SG/SF', 'Needs': '3 and D'},
    'Raptors': {'Pos_needed': 'SG/SF', 'Needs': '3 and D'},
    'Jazz': {'Pos_needed': np.nan, 'Needs': 'Young Talent/Upside'},
    'Wizards': {'Pos_needed': np.nan, 'Needs': 'Young Talent/Upside'},
    'Nets': {'Pos_needed': 'PF/C', 'Needs': 'Defensive Versatile Forward'},
    'Hornets': {'Pos_needed': 'SF/PF', 'Needs': 'Defensive Forward'},
    'Magic': {'Pos_needed': 'SG/SF', 'Needs': '3 and D'},
    'Clippers': {'Pos_needed': 'C', 'Needs': 'Defensive Center'},
    'Pistons': {'Pos_needed': 'SF/PF', 'Needs': 'Defensive Forward'},
    'Mavericks': {'Pos_needed': np.nan, 'Needs': 'Young Talent/Upside'},
    'Thunder': {'Pos_needed': 'SG/SF', 'Needs': '3 and D'},
    'Warriors': {'Pos_needed': np.nan, 'Needs': 'Young Talent/Upside'},
    'Suns': {'Pos_needed': np.nan, 'Needs': 'Young Talent/Upside'},
    'Bucks': {'Pos_needed': 'SG/SF', 'Needs': '3 and D'},
    'Pelicans': {'Pos_needed': np.nan, 'Needs': 'Shooter/Upside'},
    'Knicks': {'Pos_needed': np.nan, 'Needs': 'Young Talent/Upside'},
    'Timberwolves': {'Pos_needed': 'PG/SG', 'Needs': 'Playmaker'},
    'Cavaliers': {'Pos_needed': np.nan, 'Needs': 'Young Talent/Upside'}
}

# Previous data to help determine patterns for missing teams
historical_data = {
    'Trail Blazers': {'Pos_needed': 'PF', 'Needs': 'Raw Talent'},
    'Sacramento': {'Pos_needed': 'SF', 'Needs': 'Raw Talent/Scorer'},
    'Denver': {'Pos_needed': 'PG', 'Needs': 'NBA-ready'},
    'Golden State': {'Pos_needed': 'SG/SF', 'Needs': '3 and D'},
    'Philadelphia': {'Pos_needed': 'PG/SG', 'Needs': 'Defense/Toughness'},
    'LA Clippers': {'Pos_needed': 'PG', 'Needs': 'Shooter/Playmaker'},
    'New York': {'Pos_needed': 'PG/SG', 'Needs': 'Raw talent/Upside'},
    'LA Lakers': {'Pos_needed': 'PG', 'Needs': 'Backup PG'},
    'Memphis': {'Pos_needed': 'SF', 'Needs': 'Shooting'}
}

# Fill in the missing position needs based on patterns
missing_patterns = {
    'Young Talent/Upside': 'PG/SF',
    'Shooter/Upside': 'SG/SF',
    'Raw Talent': 'PF',
    'Raw Talent/Scorer': 'SF'
}

# Complete all teams' position needs
for team, data in current_data.items():
    if pd.isna(data['Pos_needed']) and data['Needs'] in missing_patterns:
        current_data[team]['Pos_needed'] = missing_patterns[data['Needs']]

# Add team data that might be missing from current_data
all_teams_data = {**current_data, **historical_data}

# Function to standardize team names (some inconsistencies between datasets)
def standardize_team_name(team_name):
    # Map variations to standardized names
    name_map = {
        'Sixers': 'Philadelphia',
        'LA Clippers': 'LA Clippers',
        'Lakers': 'LA Lakers',
        'Kings': 'Sacramento',
        'Nuggets': 'Denver'
    }
    return name_map.get(team_name, team_name)

# Add position needs and team needs to the draft dataframe
for i, row in draft_df.iterrows():
    team = row['Team']
    
    # Skip the forfeited pick
    if team == 'New York' and row['Original_Team'] == 'Forfeited':
        draft_df.at[i, 'Pos_needed'] = 'FORFEITED'
        draft_df.at[i, 'Needs'] = 'FORFEITED'
        continue
    
    # Standardize team name for lookup
    std_team = team
    
    # Try to find the team in our data
    if std_team in all_teams_data:
        draft_df.at[i, 'Pos_needed'] = all_teams_data[std_team]['Pos_needed']
        draft_df.at[i, 'Needs'] = all_teams_data[std_team]['Needs']
    else:
        # Try alternative names
        found = False
        for team_name, data in all_teams_data.items():
            if team.lower() in team_name.lower() or team_name.lower() in team.lower():
                draft_df.at[i, 'Pos_needed'] = data['Pos_needed']
                draft_df.at[i, 'Needs'] = data['Needs']
                found = True
                break
        
        # If still not found, set default values based on patterns
        if not found:
            draft_df.at[i, 'Pos_needed'] = 'SF/PF'  # Most common position need
            draft_df.at[i, 'Needs'] = 'Young Talent/Upside'  # Most common need

# For Portland specifically, use historical data
portland_indices = draft_df[draft_df['Team'] == 'Portland'].index
for idx in portland_indices:
    draft_df.at[idx, 'Pos_needed'] = 'PF'
    draft_df.at[idx, 'Needs'] = 'Raw Talent'

# Handle specific cases for teams where we need consistency
team_needs_consistency = {
    'Washington': {'Pos_needed': 'SF/PF', 'Needs': 'Young Talent/Upside'},
    'Charlotte': {'Pos_needed': 'SF/PF', 'Needs': 'Defensive Forward'},
    'Utah': {'Pos_needed': 'SF/PF', 'Needs': 'Young Talent/Upside'},
    'New Orleans': {'Pos_needed': 'SG/SF', 'Needs': 'Shooter/Upside'},
    'Toronto': {'Pos_needed': 'SG/SF', 'Needs': '3 and D'},
    'Philadelphia': {'Pos_needed': 'SF/PF', 'Needs': 'Defensive Stopper'},
    'Brooklyn': {'Pos_needed': 'PF/C', 'Needs': 'Defensive Versatile Forward'},
    'Chicago': {'Pos_needed': 'SF/PF', 'Needs': 'Young Talent/Upside'},
    'San Antonio': {'Pos_needed': 'SG/SF', 'Needs': '3 and D'},
    'Portland': {'Pos_needed': 'PF', 'Needs': 'Raw Talent'},
    'Houston': {'Pos_needed': 'SF/PF', 'Needs': 'Defensive Forward'},
    'Atlanta': {'Pos_needed': 'SF/PF', 'Needs': 'Young Talent/Upside'},
    'Dallas': {'Pos_needed': 'SF/PF', 'Needs': 'Young Talent/Upside'},
    'Oklahoma City': {'Pos_needed': 'SG/SF', 'Needs': '3 and D'},
    'Orlando': {'Pos_needed': 'SG/SF', 'Needs': '3 and D'},
    'Miami': {'Pos_needed': 'SF/PF', 'Needs': 'Young Talent/Upside'},
    'Minnesota': {'Pos_needed': 'PG/SG', 'Needs': 'Playmaker'},
    'Indiana': {'Pos_needed': 'SG/SF', 'Needs': '3 and D'},
    'Boston': {'Pos_needed': 'C', 'Needs': 'Defensive Center'},
    'LA Clippers': {'Pos_needed': 'C', 'Needs': 'Defensive Center'},
    'Phoenix': {'Pos_needed': 'SF/PF', 'Needs': 'Young Talent/Upside'},
    'Detroit': {'Pos_needed': 'SF/PF', 'Needs': 'Defensive Forward'},
    'Golden State': {'Pos_needed': 'SG/SF', 'Needs': '3 and D'},
    'Sacramento': {'Pos_needed': 'SF', 'Needs': 'Raw Talent/Scorer'},
    'LA Lakers': {'Pos_needed': 'PG', 'Needs': 'Backup PG'},
    'Memphis': {'Pos_needed': 'SF', 'Needs': 'Shooting'},
    'New York': {'Pos_needed': 'PG/SG', 'Needs': 'Raw talent/Upside'},
    'Cleveland': {'Pos_needed': 'SF/PF', 'Needs': 'Young Talent/Upside'}
}

# Update the dataframe for consistency
for i, row in draft_df.iterrows():
    team = row['Team']
    if team in team_needs_consistency and team != 'New York':  # Skip the forfeited NY pick
        draft_df.at[i, 'Pos_needed'] = team_needs_consistency[team]['Pos_needed']
        draft_df.at[i, 'Needs'] = team_needs_consistency[team]['Needs']

# Create cleaned dataframe for final output
final_df = draft_df[['Pick', 'Team', 'Original_Team', 'Pos_needed', 'Needs']]
final_df = final_df.rename(columns={'Team': 'Team', 'Original_Team': 'From', 'Pos_needed': 'Position Needed', 'Needs': 'Team Needs'})

# Display the final dataframe
final_df

,Pick,Team,From,Position Needed,Team Needs
0,1,Washington,Washington,SF/PF,Young Talent/Upside
1,2,Charlotte,Charlotte,SF/PF,Defensive Forward
2,3,Utah,Utah,SF/PF,Young Talent/Upside
3,4,New Orleans,New Orleans,SG/SF,Shooter/Upside
4,5,Toronto,Toronto,SG/SF,3 and D
5,6,Philadelphia,Philadelphia,SF/PF,Defensive Stopper
6,7,Brooklyn,Brooklyn,PF/C,Defensive Versatile Forward
7,8,Chicago,Chicago,SF/PF,Young Talent/Upside
8,9,San Antonio,San Antonio,SG/SF,3 and D
9,10,Portland,Portland,PF,Raw Talent


In [229]:
df.to_csv('final_df')

In [245]:
import pandas as pd

# Create the DataFrame for players and positions
df_players = pd.read_csv('/Users/ethantsao/NBADraft1/2025_draft_order_pos')

# Provided draft order and team needs
draft_order = [
    {"Pick": 1, "Team": "Washington", "Position Needed": "SF/PF", "Team Needs": "Young Talent/Upside"},
    {"Pick": 2, "Team": "Charlotte", "Position Needed": "SF/PF", "Team Needs": "Defensive Forward"},
    {"Pick": 3, "Team": "Utah", "Position Needed": "SF/PF", "Team Needs": "Young Talent/Upside"},
    {"Pick": 4, "Team": "New Orleans", "Position Needed": "SG/SF", "Team Needs": "Shooter/Upside"},
    {"Pick": 5, "Team": "Toronto", "Position Needed": "SG/SF", "Team Needs": "3 and D"},
    {"Pick": 6, "Team": "Philadelphia", "Position Needed": "SF/PF", "Team Needs": "Defensive Stopper"},
    {"Pick": 7, "Team": "Brooklyn", "Position Needed": "PF/C", "Team Needs": "Defensive Versatile Forward"},
    {"Pick": 8, "Team": "Chicago", "Position Needed": "SF/PF", "Team Needs": "Young Talent/Upside"},
    {"Pick": 9, "Team": "San Antonio", "Position Needed": "SG/SF", "Team Needs": "3 and D"},
    {"Pick": 10, "Team": "Portland", "Position Needed": "PF", "Team Needs": "Raw Talent"},
    {"Pick": 11, "Team": "Houston", "Position Needed": "SF/PF", "Team Needs": "Defensive Forward"},
    {"Pick": 12, "Team": "San Antonio", "Position Needed": "SG/SF", "Team Needs": "3 and D"},
    {"Pick": 13, "Team": "Atlanta", "Position Needed": "SF/PF", "Team Needs": "Young Talent/Upside"},
    {"Pick": 14, "Team": "Dallas", "Position Needed": "SF/PF", "Team Needs": "Young Talent/Upside"},
    {"Pick": 15, "Team": "Oklahoma City", "Position Needed": "SG/SF", "Team Needs": "3 and D"},
    {"Pick": 16, "Team": "Orlando", "Position Needed": "SG/SF", "Team Needs": "3 and D"},
    {"Pick": 17, "Team": "Utah", "Position Needed": "SF/PF", "Team Needs": "Young Talent/Upside"},
    {"Pick": 18, "Team": "Miami", "Position Needed": "SF/PF", "Team Needs": "Young Talent/Upside"},
    {"Pick": 19, "Team": "Oklahoma City", "Position Needed": "SG/SF", "Team Needs": "3 and D"},
    {"Pick": 20, "Team": "Minnesota", "Position Needed": "PG/SG", "Team Needs": "Playmaker"},
    {"Pick": 21, "Team": "Brooklyn", "Position Needed": "PF/C", "Team Needs": "Defensive Versatile Forward"},
    {"Pick": 22, "Team": "Indiana", "Position Needed": "SG/SF", "Team Needs": "3 and D"},
    {"Pick": 23, "Team": "Brooklyn", "Position Needed": "PF/C", "Team Needs": "Defensive Versatile Forward"},
    {"Pick": 24, "Team": "Atlanta", "Position Needed": "SF/PF", "Team Needs": "Young Talent/Upside"},
    {"Pick": 25, "Team": "Orlando", "Position Needed": "SG/SF", "Team Needs": "3 and D"},
    {"Pick": 26, "Team": "Washington", "Position Needed": "SF/PF", "Team Needs": "Young Talent/Upside"},
    {"Pick": 27, "Team": "Brooklyn", "Position Needed": "PF/C", "Team Needs": "Defensive Versatile Forward"},
    {"Pick": 28, "Team": "Boston", "Position Needed": "C", "Team Needs": "Defensive Center"},
    {"Pick": 29, "Team": "LA Clippers", "Position Needed": "C", "Team Needs": "Defensive Center"},
    {"Pick": 30, "Team": "Phoenix", "Position Needed": "SF/PF", "Team Needs": "Young Talent/Upside"},
    {"Pick": 31, "Team": "Boston", "Position Needed": "C", "Team Needs": "Defensive Center"},
    {"Pick": 32, "Team": "Minnesota", "Position Needed": "PG/SG", "Team Needs": "Playmaker"},
    {"Pick": 33, "Team": "Charlotte", "Position Needed": "SF/PF", "Team Needs": "Defensive Forward"},
    {"Pick": 34, "Team": "Charlotte", "Position Needed": "SF/PF", "Team Needs": "Defensive Forward"},
    {"Pick": 35, "Team": "Detroit", "Position Needed": "SF/PF", "Team Needs": "Defensive Forward"},
    {"Pick": 36, "Team": "Philadelphia", "Position Needed": "SF/PF", "Team Needs": "Defensive Stopper"},
    {"Pick": 37, "Team": "Brooklyn", "Position Needed": "PF/C", "Team Needs": "Defensive Versatile Forward"},
    {"Pick": 38, "Team": "Sacramento", "Position Needed": "SF", "Team Needs": "Raw Talent/Scorer"},
    {"Pick": 39, "Team": "San Antonio", "Position Needed": "SG/SF", "Team Needs": "3 and D"},
    {"Pick": 40, "Team": "Toronto", "Position Needed": "SG/SF", "Team Needs": "3 and D"},
    {"Pick": 41, "Team": "Oklahoma City", "Position Needed": "SG/SF", "Team Needs": "3 and D"},
    {"Pick": 42, "Team": "Washington", "Position Needed": "SF/PF", "Team Needs": "Young Talent/Upside"},
    {"Pick": 43, "Team": "Golden State", "Position Needed": "SG/SF", "Team Needs": "3 and D"},
    {"Pick": 44, "Team": "Orlando", "Position Needed": "SG/SF", "Team Needs": "3 and D"},
    {"Pick": 45, "Team": "Chicago", "Position Needed": "SF/PF", "Team Needs": "Young Talent/Upside"},
    {"Pick": 46, "Team": "LA Clippers", "Position Needed": "C", "Team Needs": "Defensive Center"},
    {"Pick": 47, "Team": "Utah", "Position Needed": "SF/PF", "Team Needs": "Young Talent/Upside"},
    {"Pick": 48, "Team": "Washington", "Position Needed": "SF/PF", "Team Needs": "Young Talent/Upside"},
    {"Pick": 49, "Team": "Utah", "Position Needed": "SF/PF", "Team Needs": "Young Talent/Upside"},
    {"Pick": 50, "Team": "Washington", "Position Needed": "SF/PF", "Team Needs": "Young Talent/Upside"},
    {"Pick": 51, "Team": "Cleveland", "Position Needed": "SF/PF", "Team Needs": "Young Talent/Upside"},
    {"Pick": 52, "Team": "Indiana", "Position Needed": "SG/SF", "Team Needs": "3 and D"},
    {"Pick": 53, "Team": "Memphis", "Position Needed": "SF", "Team Needs": "Shooting"},
    {"Pick": 54, "Team": "LA Lakers", "Position Needed": "PG", "Team Needs": "Backup PG"},
    {"Pick": 55, "Team": "Phoenix", "Position Needed": "SF/PF", "Team Needs": "Young Talent/Upside"},
    {"Pick": 56, "Team": "New York", "Position Needed": "PG/SG", "Team Needs": "Raw talent/Upside"},
    {"Pick": 57, "Team": "Orlando", "Position Needed": "SG/SF", "Team Needs": "3 and D"},
    {"Pick": 58, "Team": "Houston", "Position Needed": "SF/PF", "Team Needs": "Defensive Forward"},
    {"Pick": 59, "Team": "Cleveland", "Position Needed": "SF/PF", "Team Needs": "Defensive Forward"}
]

# Convert draft_order to DataFrame
df_draft_order = pd.DataFrame(draft_order)

# Initialize an empty list to store the matched results
matched_results = []

# Iterate through the draft order and match players
for index, row in df_draft_order.iterrows():
    pick = row['Pick']
    position_needed = row['Position Needed'].split('/')  # Split positions if multiple
    team_needs = row['Team Needs']
    
    # Find the first player in df_players whose position matches the team's needs
    for player_index, player_row in df_players.iterrows():
        if player_row['Position'] in position_needed:
            matched_results.append({
                "Pick": pick,
                "Team": row['Team'],
                "Player": player_row['Player'],
                "Position": player_row['Position'],
                "Team Needs": team_needs
            })
            df_players.drop(player_index, inplace=True)  # Remove the player from the pool
            break

# Convert the matched results to a DataFrame
df_matched = pd.DataFrame(matched_results)

# Display the matched results
df_matched

,Pick,Team,Player,Position,Team Needs
0,1,Washington,Ace Bailey,SF,Young Talent/Upside
1,2,Charlotte,Cooper Flagg,SF,Defensive Forward
2,3,Utah,Ian Schieffelin,PF,Young Talent/Upside
3,4,New Orleans,Nique Clifford,SG,Shooter/Upside
4,5,Toronto,Coleman Hawkins,SF,3 and D
5,6,Philadelphia,Collin Murray-Boyles,PF,Defensive Stopper
6,7,Brooklyn,Hansen Yang,C,Defensive Versatile Forward
7,8,Chicago,Saint Thomas,SF,Young Talent/Upside
8,9,San Antonio,Kasparas Jakucionis,SF,3 and D
9,10,Portland,Grant Nelson,PF,Raw Talent
